In [1]:
#  dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# testing beer ratings csv

df=pd.read_csv(r'data\beer_rating.csv')
df.head()


,beer name,id,beer type,Name,brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


In [14]:
bdf = df[['beer name',]]
brewery_df = bdf.groupby('brewery').count()
brewery_df.head(25)


,beer name,id,beer type,Name,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
brewery,,,,,,,,,,,,,,,,,,,,
(512) Brewing Company,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
10 Barrel Brewing Co.,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
14th Star Brewing Co.,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
18th Street Brewery - Gary Taproom,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1940's Brewing Company,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21st Amendment Brewery,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
2nd Shift Brewing,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3 Cordilleras,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3 Floyds Brewing Co.,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
